In [1]:
import pandas as pd
import plotly.express as px

In [2]:
data = pd.read_csv('accidents_data.csv', encoding='latin1', sep=';')
data.head()

,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,pessoas,mortos,feridos_graves,latitude,longitude
0,2017-01-01,domingo,00:00:00,PR,376.0,112,PARANAVAI,Fenômenos da Natureza,Queda de ocupante de veículo,Com Vítimas Feridas,Plena Noite,Crescente,Chuva,Simples,Reta,1,0,1,"-23,09880731","-52,38789369"
1,2017-01-01,domingo,19:20:00,PE,232.0,"408,9",SERRA TALHADA,Desobediência às normas de trânsito pelo condutor,Colisão transversal,Com Vítimas Feridas,Plena Noite,Decrescente,Céu Claro,Simples,Reta,2,0,2,"-8,0578381","-34,8828969"
2,2017-01-01,domingo,18:40:00,SC,116.0,"8,3",MAFRA,Não guardar distância de segurança,Colisão traseira,Com Vítimas Feridas,Pleno dia,Crescente,Nublado,Simples,Não Informado,3,0,0,"-27,5953778","-48,5480499"
3,2017-01-01,domingo,20:00:00,PR,467.0,"75,3",TOLEDO,Falta de Atenção à Condução,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Céu Claro,Dupla,Reta,1,0,0,"-24,75166889","-53,69530052"
4,2017-01-01,domingo,19:15:00,TO,153.0,"146,9",ARAGUAINA,Velocidade Incompatível,Colisão traseira,Com Vítimas Feridas,Plena Noite,Decrescente,Céu Claro,Múltipla,Não Informado,4,0,4,"-7,22108702","-48,2476294"


In [3]:
data['ano'] = data['data_inversa'].apply(lambda x: x.split('-')[0])

In [4]:
classe_acidente = data.groupby('classificacao_acidente').count()[['dia_semana']].reset_index(level=[0]).rename(columns={'data_inversa': 'data', 'dia_semana': 'quantidade_acidentes'})

In [5]:
fig = px.bar(classe_acidente, x='classificacao_acidente', y='quantidade_acidentes')
fig.show()

In [6]:
acidentes_por_ano = data.groupby(['ano', 'classificacao_acidente']).count()[['dia_semana']].reset_index(level=[0,1]).rename(columns={'dia_semana': 'quantidade_acidentes'})

In [7]:
fig = px.bar(acidentes_por_ano, x="ano", y="quantidade_acidentes", color="classificacao_acidente",
             barmode = 'stack')
 
fig.show()

In [8]:
acidentes_por_dia = data.groupby(['dia_semana', 'classificacao_acidente']).count()[['data_inversa']].reset_index(level=[0,1]).rename(columns={'data_inversa': 'quanrtidade_acidentes'})

In [9]:
fig = px.bar(acidentes_por_dia, x="dia_semana", y="quanrtidade_acidentes", color="classificacao_acidente",
             barmode = 'stack')
dias = ['domingo', 'segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira', 'sábado']
fig.update_layout(xaxis={'categoryorder': 'array', 'categoryarray': dias})
fig.show()

In [10]:
def formatar_causa(causa):
    return causa.lower()

data.loc[:, 'causa_acidente'] = data['causa_acidente'].apply(formatar_causa)
acidentes_por_causa = data.groupby(['causa_acidente', 'classificacao_acidente']).count()[['data_inversa']].reset_index(level=[0,1]).rename(columns={'data_inversa': 'quantidade_acidentes'})

In [13]:
acidentes_por_causa = acidentes_por_causa[acidentes_por_causa['quantidade_acidentes'] > 1000]
fig = px.bar(acidentes_por_causa.query('causa_acidente != "outras"'), x="quantidade_acidentes", y="causa_acidente",
             orientation='h', color="classificacao_acidente",barmode = 'stack')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [17]:
def formatar_tipo(tipo):
    tipo = tipo.lower()
    if tipo == 'Atropelamento de Pedestre':
        return 'Atropelamento de pessoa'
    elif tipo == 'Colisão com objeto em movimento':
        return 'Colisão com objeto móvel'
    elif tipo == 'Colisão com objeto fixo':
        return 'Colisão com objeto estático'
    else:
        return tipo

acidente_por_tipo = data.groupby(['tipo_acidente', 'classificacao_acidente']).count()[['data_inversa']].reset_index(level=[0, 1]).rename(columns={'data_inversa': 'quantidade_acidentes'})
acidente_por_tipo['tipo_acidente'] = acidente_por_tipo['tipo_acidente'].apply(formatar_tipo)

In [19]:
fig = px.bar(acidente_por_tipo, x="quantidade_acidentes", y="tipo_acidente",
             orientation='h', color="classificacao_acidente",barmode = 'stack')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()